In [2]:
import pandas as pd
import networkx as nx
from networkx.readwrite import json_graph
import json
import random

In [3]:
nodes_path = "../file_data/fb-pages-food.nodes"
edges_path= "../file_data/fb-pages-food.edges"

nodes_df = pd.read_csv(
    nodes_path, sep=",", header=None, names=["node", "name", "new_id"], skiprows=1
)
edges_df = pd.read_csv(edges_path, sep=",", header=None, names=["source", "target"])

nodes_df["new_id"] = nodes_df["new_id"].astype(str)
edges_df["source"] = edges_df["source"].astype(str)
edges_df["target"] = edges_df["target"].astype(str)

In [4]:
G = nx.Graph()

for index, row in nodes_df.iterrows():
    G.add_node(row["new_id"], name=row["name"], original_id=row["node"])


for index, row in edges_df.iterrows():
    G.add_edge(row["source"], row["target"])

In [5]:
max_cliques = list(nx.find_cliques(G))
print(len(max_cliques))

1249


In [6]:
def generate_community_names(num_cliques):
    prefixes = [
        "Neo",
        "Alt",
        "Hyper",
        "Meta",
        "Quantum",
        "Terra",
        "Cyber",
        "Omega",
        "Solar",
        "Astra",
    ]
    roots = [
        "Gen",
        "Net",
        "Sphere",
        "Cluster",
        "Link",
        "Wave",
        "Flow",
        "Pulse",
        "Code",
        "Echo",
    ]
    suffixes = ["ium", "on", "os", "ix", "ia", "or", "an", "is", "us", "er"]

    community_names = set()

    while len(community_names) < num_cliques:
        name = (
            f"{random.choice(prefixes)}{random.choice(roots)}{random.choice(suffixes)}"
        )
        name += f" {random.randint(1000, 9999)}"
        if(name not in community_names):
            community_names.add(name)
    print("nomi generati")
    return list(community_names)

In [7]:
def assign_clique_names_to_graph(graph, cliques):
    num_cliques = len(cliques) 
    community_names = generate_community_names(
        num_cliques
    )
    for i, clique in enumerate(cliques):
        clique_name = community_names[i]
        saved_clique = clique
        cliques[i] = {"name": f"{clique_name}" , "nodes": clique}
        for node in saved_clique:
            if not "aggregation" in graph.nodes[node]:
                graph.nodes[node]["aggregation"] = []
            graph.nodes[node]["aggregation"].append(f"{clique_name}")
            if "community_name" not in graph.nodes[node]:
                graph.nodes[node]["community_name"] = clique_name
    for edge in graph.edges:
        source = edge[0]
        graph.edges[edge]["community_name"] = graph.nodes[source]["community_name"]
    print("nomi assegnati")

In [8]:
assign_clique_names_to_graph(G, max_cliques)
for node in G.nodes(data=True):
    print(f"Node {node[0]}: {node[1]['community_name']}")

nomi generati
nomi assegnati
Node 386: MetaGenia 9195
Node 473: NeoPulseix 4566
Node 1: HyperClusterix 6531
Node 542: TerraGenia 5418
Node 189: QuantumClusterix 1257
Node 454: HyperNetos 6443
Node 158: MetaPulseer 8422
Node 248: QuantumNeton 1410
Node 537: QuantumWaveis 4206
Node 293: OmegaPulseor 7805
Node 171: OmegaClusteron 3462
Node 485: CyberEchoan 1797
Node 258: QuantumLinkan 2600
Node 534: OmegaFlower 9206
Node 500: AltWaveus 3174
Node 194: SolarWaveos 1584
Node 34: AstraLinkium 8645
Node 465: MetaPulseix 9446
Node 562: HyperNetan 3213
Node 511: OmegaSpherean 6244
Node 201: HyperEchoer 3253
Node 188: CyberCodeis 4618
Node 440: SolarGenus 9827
Node 484: SolarEchoon 5855
Node 240: QuantumGenan 6292
Node 533: HyperClusteran 2721
Node 235: NeoFlowos 8705
Node 82: AstraNeter 3823
Node 212: SolarLinkan 4991
Node 35: CyberEchoon 4442
Node 19: SolarWaveus 1897
Node 154: AltCodeus 9139
Node 334: MetaCodeia 8377
Node 148: MetaLinkia 7247
Node 599: TerraClusteros 5201
Node 382: QuantumClus

In [36]:
def create_clique_graph(graph, cliques):
    cliqueGraph = nx.Graph()
    link_map = {}  
    clique_map = {}  
    for idx, clique in enumerate(cliques):
        clique_name = clique["name"]
        cliqueGraph.add_node(
            clique_name,
            children=[node for node in clique["nodes"] if node not in clique_map],
        )
        for node in clique["nodes"]:
            clique_map[node] = clique_name
    for edge in graph.edges():
        source_node, target_node = edge

        source_clique = clique_map.get(source_node)
        target_clique = clique_map.get(target_node)
        print(source_clique, target_clique)
        if source_clique and target_clique and source_clique != target_clique:
            link_key = f"{source_clique}-{target_clique}"
            reverse_link_key = f"{target_clique}-{source_clique}"

            if link_key not in link_map and reverse_link_key not in link_map:
                cliqueGraph.add_edge(source_clique, target_clique)
                link_map[link_key] = True

    return cliqueGraph

In [37]:
cliques_graph = create_clique_graph(G, max_cliques)

{'503': 'OmegaPulseer 1627', '5': 'MetaFlowos 5887', '311': 'SolarClusteria 3892', '146': 'TerraClusteron 4981', '193': 'OmegaSphereon 5666', '567': 'NeoSphereix 9313', '93': 'OmegaGenix 7757', '498': 'CyberClusteron 7542', '253': 'OmegaClusterus 4927', '354': 'CyberGenos 5369', '505': 'OmegaClusterus 4927', '67': 'OmegaClusterus 4927', '232': 'CyberSphereor 5279', '6': 'AltGenon 7582', '518': 'NeoClusterix 3424', '163': 'AltWaveia 3821', '478': 'MetaPulseix 1470', '486': 'SolarGenix 5317', '429': 'AltSphereer 4819', '607': 'OmegaWaveon 8797', '338': 'AltGenix 8642', '435': 'MetaCodeix 8044', '203': 'MetaCodeix 8044', '345': 'MetaGenor 3506', '303': 'NeoClusteros 6003', '336': 'MetaSphereium 8154', '287': 'TerraGener 5760', '300': 'NeoGenium 6927', '506': 'QuantumGenos 2679', '418': 'QuantumNetos 2691', '323': 'QuantumCodeium 3872', '610': 'MetaPulseus 3427', '162': 'NeoFlowia 9380', '242': 'TerraCodeon 2632', '434': 'NeoClusterium 9083', '340': 'AstraLinkos 2147', '70': 'MetaGenis 827

In [38]:
graph_data = json_graph.node_link_data(G)
cliques_graph_data = json_graph.node_link_data(cliques_graph)
output_data= {
    "graph_data": graph_data,
    "max_cliques": max_cliques,
    "cliques_graph_data": cliques_graph_data
}

In [39]:
with open("../graph_data/graph_data_cliques.json", "w") as f:
    json.dump(output_data, f, indent=4)